# Benchmark rapide des algorithmes classiques de classification

Dans ce notebook, nous établissons des résultats de benchmark pour la prédiction du **sentiment** (positif ou négatif) de critiques de films, à partir d’algorithmes de machine learning classiques.

Ces résultats serviront de point de comparaison pour évaluer la performance des modèles de type **Large Language Models (LLMs)** dans le notebook suivant : `3_prediction_BERT.ipynb`.

Les étapes sont les suivantes :

### 1 - Construction de la représentation vectorielle des textes

Les algorithmes de ML classiques nécessitent que les données textuelles soient représentées sous forme de **vecteurs**. Il convient donc, dans un premier temps, de transformer les critiques de films en vecteurs numériques.

Le nombre maximal de mots par critique étant d’environ **2500 mots**, nous utilisons la méthode **bag-of-words**, qui transforme chaque avis en un vecteur de taille 2500. Cela est mis en œuvre dans `scikit-learn` avec la fonction `CountVectorizer`.

Cependant, `CountVectorizer` encode le texte à l’aide de la fréquence brute des mots, ce qui peut conduire à accorder un poids excessif aux mots fréquents, même s’ils ne sont pas discriminants. Pour remédier à cela, nous utilisons une pondération **TF-IDF (Term Frequency - Inverse Document Frequency)**, qui ajuste le poids des mots en fonction de leur fréquence dans l’ensemble des documents.

Sous `scikit-learn`, cela se fait avec la fonction `TfidfVectorizer`, qui produit directement une représentation bag-of-words pondérée par TF-IDF.


### 2 - Réduction de dimension : Latent Semantic Analysis (LSA)

Nous suivons ici une des approches décrites dans l’article *Learning Word Vectors for Sentiment Analysis* de Maas et al. (2011), en appliquant ensuite une réduction de dimensionnalité aux matrices terme-document, à l’aide de la **décomposition en valeurs singulières (SVD)**. Cette technique est également connue sous le nom de **Latent Semantic Analysis (LSA)** et permet de capturer des structures sémantiques latentes dans les données.


### 3 - Entraînement des algorithmes de classification

À partir des représentations vectorielles obtenues, nous comparons la performance de deux algorithmes de classification :

- `MultinomialNB` : un classifieur bayésien naïf, adapté aux données de type fréquence ou probabilité (comme les vecteurs TF-IDF).
- `LogisticRegression` : une régression logistique, qui se prête aux tâches de classification binaire.

Ces modèles sont entraînés sur l’ensemble `df_train`, avec une **validation croisée** pour sélectionner à partir d’une grille la valeur optimale de certains **hyperparamètres** :

- `alpha` pour le modèle `MultinomialNB`, qui contrôle le niveau de régularisation,
- `C` pour la `LogisticRegression`, qui est la valeur inverse de la régularisation.


### 4 - Évaluation des performances

Une fois les modèles entraînés, nous les évaluons sur l’ensemble `df_test`. Nous comparons les performances en générant les **matrices de confusion** et les **rapports de classification**, qui permettent d’observer la précision, le rappel, et le F1-score pour chaque classe (positif/négatif).


---

In [ ]:
"""Importation des librairies"""
# Standard library
import os
import tarfile
import urllib.request
import joblib

# Third-party libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Scikit-learn modules
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
# chargement des données d'entrainement et de test
df_train = pd.read_parquet("data/df_train.parquet")
df_test = pd.read_parquet("data/df_test.parquet")

In [ ]:
# Réduction du dataset pour rapidité
# df_small_train = df_train.iloc[:100]
# df_small_test = df_train.iloc[100:110]

X_train = df_train['texte']
y_train = df_train['label']
X_test = df_test['texte']
y_test = df_test['label']

In [7]:
"""Définition des différents pipeline"""
# Pipeline avec TF-IDF + SVD (LSA) + Logistic Regression
pipeline_logistic = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=2500)),
    ('svd', TruncatedSVD(n_components=100, random_state=42)),
    ('clf', LogisticRegression(max_iter=300))
])

# Pipeline avec TF-IDF + SVD + MultinomialNB
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('svd', TruncatedSVD(n_components=100, random_state=42)),
    ('clf', MultinomialNB())
])

In [8]:
"""Grilles de recherche pour les hyperparamètres lors de la validation croisée"""
# Grille de recherche pour la régression logistique
param_grid_logistic = {
    'clf__C': [0.01, 0.1, 1, 10]
}

# Grille plus simple pour le NB (alpha = lissage)
param_grid_nb = {
    'clf__alpha': [0.5, 1.0, 1.5]
}

In [10]:
"""Entraînement du modèle de régression logistique"""
grid_logistic = GridSearchCV(pipeline_logistic, param_grid_logistic, cv=3, scoring='accuracy')
grid_logistic.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=2500,
                                                        stop_words='english')),
                                       ('svd',
                                        TruncatedSVD(n_components=100,
                                                     random_state=42)),
                                       ('clf',
                                        LogisticRegression(max_iter=300))]),
             param_grid={'clf__C': [0.01, 0.1, 1, 10]}, scoring='accuracy')

In [ ]:
"""Enregistrement du modèle entraîné"""
model_filename = "models/logistic_regression_tfidf_svd_gridsearch.joblib"
joblib.dump(grid_logistic, model_filename)
print(f"Modèle enregistré sous : {model_filename}")

Modèle enregistré sous : models/logistic_regression_tfidf_svd_gridsearch.joblib


In [16]:
"""Évaluation du modèle de régression"""

# Chargement du modèle
loaded_model = joblib.load("models/logistic_regression_tfidf_svd_gridsearch.joblib")

# Réaliser les prédictions sur l'ensemble de test
y_pred_logistic = loaded_model.predict(X_test)

# Afficher les résultats
print("Logistic Regression - Best Params:", grid_logistic.best_params_)
y_pred_logistic = grid_logistic.predict(X_test)
print("Classification Report - Logistic Regression:\n", classification_report(y_test, y_pred_logistic))
print("Confusion Matrix - Logistic Regression:\n", confusion_matrix(y_test, y_pred_logistic))

Logistic Regression - Best Params: {'clf__C': 10}
Classification Report - Logistic Regression:
               precision    recall  f1-score   support

           0       0.86      0.85      0.85     12500
           1       0.85      0.86      0.85     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

Confusion Matrix - Logistic Regression:
 [[10564  1936]
 [ 1757 10743]]


In [ ]:
"""Entraînement du modèle avec classifier de Bayes"""
grid_nb = GridSearchCV(pipeline_nb, param_grid_nb, cv=3, scoring='accuracy')
grid_nb.fit(X_train, y_train)